# Visualizing Training Statistics with Tensorborad
PyTorch integrates with TensorBoard, a tool designed for visualizing the results of neural network training runs.

In this tutorial, we’ll learn how to:

- Set up TensorBoard.

- Write to TensorBoard.

- Inspect a model architecture using TensorBoard.

- Use TensorBoard to create interactive versions of the statistics visualizations (like loss, accuracy, etc..)

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    img = img.cpu()
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    
    to_pil = transforms.ToPILImage()
    
    if one_channel:
        npimg = img.numpy()
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(to_pil(img))

In [ ]:
# let's define a model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net = Net().to(device)

In [ ]:
# Optimizer and loss function
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

Now we’ll set up TensorBoard, importing tensorboard from <code>torch.utils</code> and defining a <code>SummaryWriter</code>, our key object for writing information to TensorBoard.

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('./runs/fashion_mnist_experiment_1')

Now let’s write an image to our TensorBoard - specifically, a grid using <code>make_grid</code>.

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_image('four_fashion_mnist_images', img_grid)

### Now lets run:
<code>tensorboard --logdir=runs</code>

One of TensorBoard’s strengths is its ability to visualize complex model structures. Let’s visualize the model we built.

In [ ]:
writer.add_graph(net, images.to(device))
writer.close()

## Tracking Model Training

In [ ]:
# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 12))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

Finally, let’s train the model using the usual model training code, but writing results to TensorBoard every 1000 batches instead of printing to console; this is done using the <code>add_scalar</code> function.

In [ ]:
from matplotlib.lines import Line2D

# get gradients list for each layer in the network
def add_gradient_hist(net):
    ave_grads = [] 
    layers = []
    for n,p in net.named_parameters():
        if ("bias" not in n):
            layers.append(n)
            if p.requires_grad: 
                ave_grad = np.abs(p.grad.clone().detach().cpu().numpy()).mean()
            else:
                ave_grad = 0
            ave_grads.append(ave_grad)
        
    layers = [layers[i].replace(".weight", "") for i in range(len(layers))]
    
    fig = plt.figure(figsize=(12, 12))
    plt.bar(np.arange(len(ave_grads)), ave_grads, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads) + 1, lw=2, color="k")
    plt.xticks(range(0, len(ave_grads), 1), layers, rotation=90)
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom=-0.001, top=np.max(ave_grads) / 2)  # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    #plt.grid(True)
    plt.legend([Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['mean-gradient', 'zero-gradient'])
    plt.tight_layout()
    #plt.show()
    
    return fig

        

        

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

running_loss = 0.0
correct = 0.0

# define a function to freeze the model layers
def set_conv_parameter_requires_grad(model, req_grad = False):
    for n,p in net.named_parameters():
        if "conv" in n:
            p.requires_grad = req_grad

# set_conv_parameter_requires_grad(net)

for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # put data to device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # ...log a Matplotlib Figure showing the model's gradients fo each layer
        if i % 1000 == 999:
            writer.add_figure('gradients',
                            add_gradient_hist(net),
                            global_step=epoch * len(trainloader) + i)

        optimizer.step()

        running_loss += loss.item()
        
        correct += get_num_correct(outputs, labels)
        if i % 1000 == 999:    # every 1000 mini-batches...

            # ...log the running loss
            writer.add_scalar('training loss',
                            running_loss / 1000,
                            epoch * len(trainloader) + i)
            # ...log the training accuracy
            writer.add_scalar('training accuracy',
                            correct / (1000*inputs.size(0)), # inputs.size(0) is the batch size
                            epoch * len(trainloader) + i)
            
            print('It: {} Loss: {} Accuracy: {}'.format(epoch * len(trainloader) + i, running_loss / 1000, correct / (1000*inputs.size(0))))

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_figure('predictions vs. actuals',
                            plot_classes_preds(net, inputs, labels),
                            global_step=epoch * len(trainloader) + i)
            
            running_loss = 0.0
            correct = 0.0
writer.close()
print('Finished Training')

# Assessing trained models with TensorBoard
Let's calculate the precision recall curve of our model.

REMINDER:

![alt text](https://miro.medium.com/max/720/0*eBNoU76LKUuCS6ap.png)

Precision $P$ is defined as the number of true positives ($T_p$) over the number of true positives plus the number of false positives ($F_p$). Precision can be thought of as the fraction of positive predictions that actually belong to the positive class:
$$P = \frac{T_p}{T_p + F_p}$$

Recall $R$  is defined as the number of true positives ($T_p$) over the number of true positives plus the number of false negatives ($F_n$). Recall can be thought of as the fraction of positive predictions out of all positive instances in the data set:
$$R = \frac{T_p}{T_p + F_n}$$

The precision-recall curve shows the tradeoff between precision and recall for different threshold. A high area under the curve represents both high recall and high precision, where high precision relates to a low false positive rate, and high recall relates to a low false negative rate. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall).

In [ ]:
# 1. gets the probability predictions in a test_size x num_classes Tensor
# 2. gets the preds in a test_size Tensor
# takes ~10 seconds to run
class_probs = []
class_label = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        output = net(images)
        class_probs_batch = [F.softmax(el.cpu(), dim=0) for el in output]

        class_probs.append(class_probs_batch)
        class_label.append(labels.cpu())

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_label = torch.cat(class_label)

# helper function
def add_pr_curve_tensorboard(class_index, test_probs, test_label, global_step=0):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# plot all the pr curves
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_label)